# Create boundary conditions

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import datetime
from mpl_toolkits.basemap import Basemap, cm
import cmocean
import glob
from math import isclose
import scipy.ndimage as ndimage
import matplotlib
import pickle
import xarray as xr

%matplotlib inline

#### Parameters:

In [2]:
# domain dimensions:
imin, imax = 1479, 2179
jmin, jmax = 159, 799

# Boundary coordinates:
N_coordinates = ((2168, 2178, 201, 798))
E_coordinates = ((1481, 2178, 778, 798))
S_coordinates = ((1482, 1492, 181, 798))

# Colours:
land_color = '#a9a7a2'

# Rimwidth:
rimwidth_north = 10 # western Canada Basin
rimwidth_east  = 20 # northern Canada Basin
rimwidth_south = 10 # Baffin Bay

#### Functions:

In [122]:
def flatten_input(var, order):
    b = var[0,:,:].flatten(order=order)
    for i in range(1,len(var)):
        a = var[i,:,:].flatten(order=order)
        b = np.vstack((b,a))
    return b

In [7]:
def create_boundary(rimwidth, boundary_dMn, boundary_oMn):

    # Take in the boundary values and reshape
    
    dMn_O = flatten_input(boundary_dMn)
    oMn_O = flatten_input(boundary_oMn)
    
    dMn_OBC = np.reshape(dMn_O, (1,50,1,np.max(boundary_dMn.shape)*rimwidth))
    oMn_OBC = np.reshape(oMn_O, (1,50,1,np.max(boundary_dMn.shape)*rimwidth))

    return dMn_OBC, oMn_OBC

In [8]:
def save_BC_file(name, dMn_BC, oMn_BC):
    
    # Save boundary conditions to file
    ncd = nc.Dataset(f'/ocean/brogalla/GEOTRACES/data/{name}', 'w', zlib=True)
    ncd.createDimension('x', np.max(dMn_BC.shape))
    ncd.createDimension('y',1)
    ncd.createDimension('z',50)
    ncd.createDimension('t',None)
    
    dMn = ncd.createVariable('dMn', 'float64', ('t','z','y','x'))
    dMn.units = 'dissolved Mn'
    dMn.long_name = 'dMn'
    dMn[:] = dMn_BC

    oMn = ncd.createVariable('oMn', 'float64', ('t','z','y','x'))
    oMn.units = 'oxidised Mn'
    oMn.long_name = 'oMn'
    oMn[:] = oMn_BC
    
    ncd.close()
    return

#### Load files:

In [126]:
ref       = nc.Dataset('/data/brogalla/ANHA12/ANHA12-EXH006_5d_gridT_y2015m01d05.nc',  'r')
lat_model = np.array(ref.variables['nav_lat'])
lon_model = np.array(ref.variables['nav_lon'])

In [127]:
# ANHA12 mesh:
mesh         = nc.Dataset('/ocean/brogalla/GEOTRACES/data/ANHA12/ANHA12_mesh1.nc')
mesh_lon     = np.array(mesh.variables['nav_lon'])
mesh_lat     = np.array(mesh.variables['nav_lat'])
mesh_bathy   = np.array(mesh.variables['tmask'][0,:,:,:])
bathy_masked = np.ma.masked_where((mesh_bathy> 0.1), mesh_bathy)
mesh_depth   = np.array(mesh.variables['nav_lev'])
mesh_hdept   = np.array(mesh.variables['hdept'][0])

#### Calculations and Figures:

In [ ]:
# then run for 4 years and use grid cells within domain for the boundary condition.
month = 5

# Load spin up run:
Mn_run = nc.Dataset('/data/brogalla/run_storage/Mn-IC-202204/ANHA12_IC-spin-up3_20220419/ANHA12_EXH006_2002_monthly.nc')
dMn = np.array(Mn_run.variables['dissolmn'])[:,0,:,:,:]
oMn = np.array(Mn_run.variables['oxidismn'])[:,0,:,:,:]
# Mask values on land
dMn_ma = np.ma.masked_where(mesh_bathy[:,imin:imax,jmin:jmax] < 0.1, dMn[month,:,:,:])
oMn_ma = np.ma.masked_where(mesh_bathy[:,imin:imax,jmin:jmax] < 0.1, oMn[month,:,:,:])

# ---------- Estimate boundary conditions based on rimwidth grid cells inside the boundary -------------
# ---- North ----
dMn_north_ma = dMn_ma[:,N_coordinates[0]-imin-rimwidth_north:N_coordinates[1]-imin-rimwidth_north,N_coordinates[2]-jmin:N_coordinates[3]-jmin]
oMn_north_ma = oMn_ma[:,N_coordinates[0]-imin-rimwidth_north:N_coordinates[1]-imin-rimwidth_north,N_coordinates[2]-jmin:N_coordinates[3]-jmin]
# Fill area of overlap with eastern boundary with average of masked array
dMn_north_ma[:,:,-20:] = np.ma.average(dMn_north_ma[:,:,-20:])
oMn_north_ma[:,:,-20:] = np.ma.average(dMn_north_ma[:,:,-20:])

# ---- East ----
dMn_east_ma  = dMn_ma[:,E_coordinates[0]-imin:E_coordinates[1]-imin,E_coordinates[2]-jmin-rimwidth_east:E_coordinates[3]-jmin-rimwidth_east]
oMn_east_ma  = oMn_ma[:,E_coordinates[0]-imin:E_coordinates[1]-imin,E_coordinates[2]-jmin-rimwidth_east:E_coordinates[3]-jmin-rimwidth_east]
# Fill area of overlap with northern boundary with average of masked array
dMn_east_ma[:,-10:,:] = np.ma.average(dMn_east_ma[:,0:-10,:])
oMn_east_ma[:,-10:,:] = np.ma.average(dMn_east_ma[:,0:-10,:])

# ---- South ----
dMn_south_ma = dMn_ma[:,S_coordinates[0]-imin+rimwidth_south:S_coordinates[1]-imin+rimwidth_south,S_coordinates[2]-jmin:S_coordinates[3]-jmin]
oMn_south_ma = oMn_ma[:,S_coordinates[0]-imin+rimwidth_south:S_coordinates[1]-imin+rimwidth_south,S_coordinates[2]-jmin:S_coordinates[3]-jmin]

for z in range(0,50):
    # Fill land with average values in slice
    dMn_north_ma[z,:,:].filled(fill_value=np.ma.average(dMn_north_ma[z,:,:]))
    oMn_north_ma[z,:,:].filled(fill_value=np.ma.average(oMn_north_ma[z,:,:]))
    dMn_east_ma[z,:,:].filled(fill_value=np.ma.average(dMn_east_ma[z,:,:]))
    oMn_east_ma[z,:,:].filled(fill_value=np.ma.average(oMn_east_ma[z,:,:]))
    dMn_south_ma[z,:,:].filled(fill_value=np.ma.average(dMn_south_ma[z,:,:]))
    oMn_south_ma[z,:,:].filled(fill_value=np.ma.average(oMn_south_ma[z,:,:]))
    
    # Mackenzie model forcing copied over, so fill with average of masked array
    dMn_north_ma[z,:,0:300] = np.ma.average(dMn_north_ma[z,:,300:])
    oMn_north_ma[z,:,0:300] = np.ma.average(oMn_north_ma[z,:,300:])
    
    # Inner Nares Strait copied over, so fill with average of masked array
    dMn_east_ma[z,0:170,:] = np.ma.average(dMn_east_ma[z,170:,:])
    oMn_east_ma[z,0:170,:] = np.ma.average(oMn_east_ma[z,170:,:])
    
dMn_north = np.array(dMn_north_ma); oMn_north = np.array(oMn_north_ma);
dMn_east  = np.array(dMn_east_ma);  oMn_east  = np.array(oMn_east_ma);
dMn_south = np.array(dMn_south_ma); oMn_south = np.array(oMn_south_ma);

# Replace zero values with average values:
dMn_north[dMn_north == 0] = np.ma.average(dMn_north_ma[:,:,:])
oMn_north[oMn_north == 0] = np.ma.average(oMn_north_ma[:,:,:])
dMn_east[dMn_east == 0]   = np.ma.average(dMn_east_ma[:,:,:])
oMn_east[oMn_east == 0]   = np.ma.average(oMn_east_ma[:,:,:])
dMn_south[dMn_south == 0] = np.ma.average(dMn_south_ma[15:,:,:]) # ignore surface otherwise fill values uncharacteristic
oMn_south[oMn_south == 0] = np.ma.average(oMn_south_ma[15:,:,:])

# Filter to smoothe out hard edges in the forcing associated with copied over land
dMn_north = ndimage.gaussian_filter(dMn_north, sigma=4, order=0)
oMn_north = ndimage.gaussian_filter(oMn_north, sigma=4, order=0)
dMn_east  = ndimage.gaussian_filter(dMn_east , sigma=4, order=0)
oMn_east  = ndimage.gaussian_filter(oMn_east , sigma=4, order=0)
dMn_south = ndimage.gaussian_filter(dMn_south, sigma=4, order=0)
oMn_south = ndimage.gaussian_filter(oMn_south, sigma=4, order=0)
    
# Flatten boundaries
dMn_north_BC, oMn_north_BC = create_boundary(rimwidth_north, dMn_north, oMn_north, 'C')
dMn_east_BC , oMn_east_BC  = create_boundary(rimwidth_east , dMn_east , oMn_east , 'F')
dMn_south_BC, oMn_south_BC = create_boundary(rimwidth_south, dMn_south, oMn_south, 'C')

#### Write final boundary condition:

In [137]:
save_BC_file('Mn_North_OBC_20220422.nc', dMn_north_BC, oMn_north_BC)
save_BC_file('Mn_East_OBC_20220422.nc' , dMn_east_BC,  oMn_east_BC)
save_BC_file('Mn_South_OBC_20220422.nc', dMn_south_BC, oMn_south_BC)

In [ ]:
print(dMn_north.size-np.count_nonzero(dMn_north))
print(oMn_north.size-np.count_nonzero(oMn_north))
print(dMn_east.size-np.count_nonzero(dMn_east))
print(oMn_east.size-np.count_nonzero(oMn_east))
print(dMn_south.size-np.count_nonzero(dMn_south))
print(oMn_south.size-np.count_nonzero(oMn_south))